In [1]:
import os
os.getcwd()
os.chdir('..')

In [2]:
%load_ext autoreload
%autoreload 2

import skimage.io as io
import PIL.Image
from IPython.display import Image 

from knowgraph_conceptnet import KnowledgeGraph
from data.utils import *
import numpy as np
from scipy import spatial

from transformers import CLIPProcessor, CLIPTextModel, CLIPTokenizer, CLIPVisionModel, CLIPModel
import pickle
import torch
from transformers import AutoTokenizer
import time
import h5py

In [3]:
from models.transformer import Transformer, MemoryAugmentedEncoder, PromptDecoder, ScaledDotProductAttentionMemory, MultiLevelEncoder, ScaledDotProductAttention, VanillaDecoder, ParallelPromptDecoder , StackedPromptDecoder
from knowgraph_conceptnet import KnowledgeGraph

In [6]:
encoder = MemoryAugmentedEncoder(1, 0, d_in=728,  attention_module=ScaledDotProductAttention,
                                     attention_module_kwargs={'m': 0}, dropout=0.1, d_model = 512, h=8)

# seg_token = args.seg_token == "True"

# knowledge_graph = KnowledgeGraph(transform_tok = tokenizerBW_clip, device = device, edge_select=args.edge_select, spec = spec, kw_size = args.num_keywords, rw_size = args.num_relatedwords , enc_model = args.enc_model, only_kw=args.only_kw, norel= args.no_rel_label, only_l2r = args.rel_only_l2r, use_faiss = args.use_faiss, rc_posidx2 =args.rc_posidx2, cn_version=args.cn_version)
knowledge_graph= None
max_inp_seq= 128
if args.decoder == "kg_infused":
    print("using normal dec")
    decoder = PromptDecoder(len(tokenizerBW), max_inp_seq, args.N_dec, spec['pad_tokenid'],h=args.head, seg_token= seg_token, KG = knowledge_graph , enc_model= args.enc_model, spec=spec, pt_tokemb=args.pt_token_emb, dropout=args.dropout, d_model = args.d_model, seg_param=args.seg_param)
elif args.decoder == "parallel":
    print("using parallel dec")
    decoder = ParallelPromptDecoder(len(tokenizerBW), max_inp_seq, args.N_dec, spec['pad_tokenid'], h=args.head, seg_token= seg_token, KG = knowledge_graph , enc_model= args.enc_model, spec=spec, pt_tokemb=args.pt_token_emb, dropout=args.dropout, d_model = args.d_model, pll_dec_type = args.pll_dec, seg_token_kw = args.seg_token_kw, seg_param=args.seg_param)
elif args.decoder == "stacked":
    print("using stacked decoder")
    decoder = StackedPromptDecoder(len(tokenizerBW), max_inp_seq, args.N_dec, spec['pad_tokenid'], h=args.head, seg_token= seg_token, KG = knowledge_graph , enc_model= args.enc_model, spec=spec, pt_tokemb=args.pt_token_emb, dropout=args.dropout, one_kw_token=args.one_kw_token, d_model = args.d_model, seg_token_kw = args.seg_token_kw, use_gpt=args.stck_gpt2, seg_param=args.seg_param)
elif args.decoder == "vanilla":
   print("using vanilla decoder")
   decoder = VanillaDecoder(len(tokenizerBW), max_inp_seq, args.N_dec, spec['pad_tokenid'], h=args.head, enc_model = args.enc_model, dropout=args.dropout, d_model = args.d_model, seg_token_kw = args.seg_token_kw, seg_param=args.seg_param)

model = Transformer(spec['bos_tokenid'], encoder, decoder).to(device)


NameError: name 'args' is not defined